In [1]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta, datetime

def generate_dataset(num_subjects=10, min_timesteps=10, max_timesteps=20, seed=37):
    random.seed(seed)
    np.random.seed(seed)
    all_rows = []                     
    true_changepoints = {} 
    # Define CPTs (Conditional Probability Tables)
    
    # Segment 1: Mapping from blood_pressure_t-1 to pain_level
    pain_cpt_seg1 = {
        0: [0.8, 0.1, 0.1],  # low BP → no pain
        1: [0.1, 0.8, 0.1],  # normal BP → mild pain
        2: [0.1, 0.1, 0.8],  # high BP → severe pain
    }

    # Segment 2: Noisier mapping from blood_pressure_t-1 to pain_level
    pain_cpt_seg2 = {
        0: [0.4, 0.3, 0.3],
        1: [0.3, 0.4, 0.3],
        2: [0.3, 0.3, 0.4],
    }

    # Segment 1: pain_level_t-1 predicts mobility
    mobility_cpt_seg1 = {
        0: [0.9, 0.1],  # no pain → walking
        1: [0.4, 0.6],  # mild pain → mostly walking
        2: [0.2, 0.8],  # severe pain → mostly bedridden
    }

    # Segment 2: Noisier mapping between pain_level_t-1 and mobility
    mobility_cpt_seg2 = {
        0: [0.6, 0.4],
        1: [0.5, 0.5],
        2: [0.8, 0.2],
    }

    # Generate data per subject
    for subject_id in range(1, num_subjects + 1):
        # Random number of time steps per subject
        num_timesteps = random.randint(min_timesteps, max_timesteps)

        # Define the changepoint (midpoint of time steps)
        changepoint = num_timesteps // 2
        true_changepoints[subject_id] = changepoint

        # Generate time series dates for the subject
        start_date = datetime(2023, random.randint(1, 12), random.randint(1, 28))
        timestamps = [start_date + timedelta(days=2 * i) for i in range(num_timesteps)]

        # Initialize first values randomly
        bp = [random.choice([0, 1, 2])]  # blood_pressure
        pain = [random.choice([0, 1, 2])]
        mobility = [random.choice([0, 1])]
        hr = [random.choice([0, 1, 2])]
        ox = [random.choice([0, 1])]

        # Generate remaining values using CPTs
        for t in range(1, num_timesteps):
            bp.append(random.choice([0, 1, 2]))

            # Use CPTs depending on segment (before/after changepoint)
            if t < changepoint:
                pain_probs = pain_cpt_seg1[bp[t - 1]]
                mobility_probs = mobility_cpt_seg1[pain[t - 1]]
            else:
                pain_probs = pain_cpt_seg2[bp[t - 1]]
                mobility_probs = mobility_cpt_seg2[pain[t - 1]]

            # Sample next values using the CPTs
            pain.append(np.random.choice([0, 1, 2], p=pain_probs))
            mobility.append(np.random.choice([0, 1], p=mobility_probs))
            hr.append(random.choice([0, 1, 2]))   
            ox.append(random.choice([0, 1]))      

        # Collect all rows for this subject
        for t in range(num_timesteps):
            all_rows.append({
                'subject_id': subject_id,
                'timestamp': timestamps[t].date(),
                'blood_pressure': bp[t],
                'pain_level': pain[t],
                'mobility': mobility[t],
                'heart_rate': hr[t],
                'oxygen_level': ox[t],
            })

    # Create a DataFrame and sort for proper ordering
    df = pd.DataFrame(all_rows)
    df = df.sort_values(by=['subject_id', 'timestamp']).reset_index(drop=True)

    return df, true_changepoints

# Generate the dataset and print the head
df_health, ground_truth_cp = generate_dataset()
subject_lengths = df_health.groupby("subject_id").size().to_dict()
df_health.head()


,subject_id,timestamp,blood_pressure,pain_level,mobility,heart_rate,oxygen_level
0,1,2023-10-03,2,2,0,2,1
1,1,2023-10-05,1,2,1,2,0
2,1,2023-10-07,2,1,1,1,1
3,1,2023-10-09,1,2,1,2,1
4,1,2023-10-11,1,1,1,0,0


In [ ]:
# Group by subject_id and count rows
subject_lengths = df_health.groupby("subject_id").size()
print("Number of rows per subject:")
for sid, length in subject_lengths.items():
    print(f"Subject {sid}: {length} time steps")


Number of rows per subject:
Subject 1: 20 time steps
Subject 2: 14 time steps
Subject 3: 19 time steps
Subject 4: 12 time steps
Subject 5: 17 time steps
Subject 6: 15 time steps
Subject 7: 20 time steps
Subject 8: 11 time steps
Subject 9: 14 time steps
Subject 10: 13 time steps


In [2]:
# Preprocessing Step: Add Lagged Variables and Encode Categories
# Function to create lagged (t-1) variables per subject
from sklearn.preprocessing import LabelEncoder
def create_lagged_df(df, id_col='subject_id', time_col='timestamp'):
    """
    For each subject:
    - Sorts data by timestamp
    - Adds lagged (t-1) versions of all feature columns
    - Returns a combined dataframe with current and lagged variables
    """
    all_lagged_rows = []

    for subject_id, group in df.groupby(id_col):
        group = group.sort_values(by=time_col).reset_index(drop=True)

        # Create lagged version (shift down by 1)
        lagged = group.shift(1)

        # Rename lagged columns to include "_t-1", except id and timestamp
        lagged.columns = [f"{col}_t-1" if col not in [id_col, time_col] else col for col in lagged.columns]

        # Combine original and lagged data side-by-side
        combined = pd.concat([group, lagged], axis=1).dropna().reset_index(drop=True)

        # Drop duplicate columns like subject_id and timestamp if they exist twice
        combined = combined.loc[:, ~combined.columns.duplicated()]

        all_lagged_rows.append(combined)

    return pd.concat(all_lagged_rows).reset_index(drop=True)

# Function to encode categorical columns using LabelEncoder

def encode_categorical_columns(df, exclude_cols=['subject_id', 'timestamp']):
    
    df_encoded = df.copy()
    encoders = {}

    for col in df.columns:
        if col in exclude_cols:
            continue
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df[col])
        encoders[col] = le

    return df_encoded, encoders

# Run Preprocessing

df_lagged = create_lagged_df(df_health)
df_encoded, encoders = encode_categorical_columns(df_lagged)
print("Original (lagged) columns:", df_lagged.columns.tolist())
print("Encoded data sample:")
print(df_encoded.head())


Original (lagged) columns: ['subject_id', 'timestamp', 'blood_pressure', 'pain_level', 'mobility', 'heart_rate', 'oxygen_level', 'blood_pressure_t-1', 'pain_level_t-1', 'mobility_t-1', 'heart_rate_t-1', 'oxygen_level_t-1']
Encoded data sample:
   subject_id   timestamp  blood_pressure  pain_level  mobility  heart_rate  \
0           1  2023-10-05               1           2         1           2   
1           1  2023-10-07               2           1         1           1   
2           1  2023-10-09               1           2         1           2   
3           1  2023-10-11               1           1         1           0   
4           1  2023-10-13               0           1         1           1   

   oxygen_level  blood_pressure_t-1  pain_level_t-1  mobility_t-1  \
0             0                   2               2             0   
1             1                   1               2             1   
2             1                   2               1             1   
3    

In [3]:
# Step 3: BDe Score
from collections import defaultdict
from scipy.special import gammaln
import numpy as np

# Computes the BDe score for a single node (child variable)
# given its parent variables using the Dirichlet prior.
def compute_bde_score(df, child, parents, alpha=1.0):
    """
    Uses the exact BDe formula (log-marginal likelihood over Dirichlet prior)
    """
    r = df[child].nunique()  # Number of unique states for the child
    q = 1  # Number of parent configurations (defaults to 1 if no parents)

    if parents:
        # Compute number of total parent configurations (cardinality product)
        q = np.prod([df[p].nunique() for p in parents])
        grouped = df.groupby(parents)
    else:
        grouped = [((), df)]

    score = 0.0
    alpha_per_value = alpha / r  # Assume uniform Dirichlet prior

    for _, group in grouped:
        counts = group[child].value_counts().to_dict()
        Nij = sum(counts.values())
        term = gammaln(alpha) - gammaln(alpha + Nij)
        for k in range(r):
            Nijk = counts.get(k, 0)
            term += gammaln(alpha_per_value + Nijk) - gammaln(alpha_per_value)
        score += term
    return score

# Computes the total BDe score for a graph structure
def compute_bde_score_for_graph(df, graph, alpha=1.0):
    total_score = 0.0
    for child, parents in graph.items():
        total_score += compute_bde_score(df, child, parents, alpha)
    return total_score

graph_true = {
    'pain_level': ['blood_pressure_t-1'],    # true edge
    'mobility': ['pain_level_t-1'],          # true edge
    'blood_pressure': [],
    'heart_rate': [],
    'oxygen_level': [],
    'blood_pressure_t-1': [],
    'pain_level_t-1': [],
    'mobility_t-1': [],
    'heart_rate_t-1': [],
    'oxygen_level_t-1': []
}

score_true = compute_bde_score_for_graph(df_encoded, graph_true)
score_empty = compute_bde_score_for_graph(df_encoded, {
    node: [] for node in df_encoded.columns if node not in ['subject_id', 'timestamp']
})

print("BDe Score Comparison:")
print("Score with true graph:  ", score_true)
print("Score with empty graph: ", score_empty)
print("Difference (true - empty):", score_true - score_empty)


BDe Score Comparison:
Score with true graph:   -1372.4736720260603
Score with empty graph:  -1394.4686811783158
Difference (true - empty): 21.995009152255534


In [ ]:
# RJMCMC per subjec

import random
import copy
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from scipy.special import gammaln

# BDe Local Score
def compute_bde_score(df, child, parents, alpha=1.0):
    r = df[child].nunique()
    grouped = df.groupby(parents) if parents else [((), df)]

    score = 0.0
    alpha_per_value = alpha / r

    for _, group in grouped:
        counts = group[child].value_counts().to_dict()
        Nij = sum(counts.values())

        term = gammaln(alpha) - gammaln(alpha + Nij)
        for k in range(r):
            Nijk = counts.get(k, 0)
            term += gammaln(alpha_per_value + Nijk) - gammaln(alpha_per_value)
        score += term

    return score

#Graph Neighbors
def get_graph_neighbors(graph, nodes, max_parents=3):
    neighbors = []

    for child in nodes:
        current_parents = set(graph.get(child, []))

        for candidate in nodes:
            if candidate == child or candidate in current_parents:
                continue
            if len(current_parents) >= max_parents:
                continue
            if "_t-1" not in candidate:
                continue

            new_graph = copy.deepcopy(graph)
            new_graph[child] = list(current_parents | {candidate})
            neighbors.append(new_graph)

        for parent in current_parents:
            new_graph = copy.deepcopy(graph)
            new_graph[child] = list(current_parents - {parent})
            neighbors.append(new_graph)

    return neighbors

# Changepoint RJMCMC Support
def initialize_changepoints(df, variables):
    V = {}
    T = df.shape[0]
    for var in variables:
        V[var] = np.zeros(T, dtype=int)
    return V

def segment_data(df, Vn):
    segments = {}
    for seg_id in np.unique(Vn):
        idx = np.where(Vn == seg_id)[0]
        segments[seg_id] = df.iloc[idx]
    return segments

def score_segments_bde(df, Vn, child, parents, alpha=1.0):
    segments = segment_data(df, Vn)
    total_score = 0.0
    for seg_df in segments.values():
        if seg_df.shape[0] == 0:
            continue
        total_score += compute_bde_score(seg_df, child, parents, alpha)
    return total_score

def changepoint_log_prior(Vn, lam=1.0):
    num_cp = sum(Vn[i] != Vn[i - 1] for i in range(1, len(Vn)))
    return -lam * num_cp

def birth_move(Vn):
    T = len(Vn)
    valid_points = [t for t in range(1, T - 1) if Vn[t] == Vn[t - 1]]
    if not valid_points:
        return Vn, False

    new_cp = random.choice(valid_points)
    new_Vn = Vn.copy()
    curr_seg = Vn[new_cp]
    new_seg_id = max(Vn) + 1

    for t in range(new_cp, T):
        if Vn[t] == curr_seg:
            new_Vn[t] = new_seg_id
        else:
            break

    return new_Vn, True

def death_move(Vn):
    unique_segs, counts = np.unique(Vn, return_counts=True)
    if len(unique_segs) <= 1:
        return Vn, False

    merge_seg = random.choice(unique_segs[1:])
    prev_seg = merge_seg - 1

    new_Vn = Vn.copy()
    new_Vn[Vn == merge_seg] = prev_seg
    _, new_vals = np.unique(new_Vn, return_inverse=True)
    return new_vals, True

def reallocation_move(Vn):
    T = len(Vn)
    change_points = [t for t in range(1, T) if Vn[t] != Vn[t - 1]]
    if len(change_points) == 0:
        return Vn, False

    cp = random.choice(change_points)
    shift = random.choice([-1, 1])
    new_cp = cp + shift

    if new_cp <= 0 or new_cp >= T:
        return Vn, False

    new_Vn = Vn.copy()
    curr_seg = Vn[cp]
    prev_seg = Vn[cp - 1]

    if shift == -1:
        new_Vn[new_cp:cp] = curr_seg
        new_Vn[new_cp] = prev_seg
    else:
        new_Vn[cp:new_cp] = prev_seg
        new_Vn[cp] = curr_seg

    return new_Vn, True

# Run RJMCMC Per Subject with BMA
def run_rjmcmc_per_subject(df, nodes, **kwargs):
    full_best_V = {node: np.zeros(len(df), dtype=int) for node in nodes}
    edge_counts = Counter()
    total_graph_samples = 0

    for sid, group in df.groupby("subject_id"):
        print(f"RJMCMC for subject {sid}...")
        local_df = group.reset_index(drop=True)
        local_nodes = [col for col in local_df.columns if col not in ['subject_id', 'timestamp']]

        G = {node: [] for node in local_nodes}
        V = initialize_changepoints(local_df, local_nodes)

        def total_score(G, V):
            score = 0.0
            for n in local_nodes:
                score += score_segments_bde(local_df, V[n], n, G[n], kwargs['alpha'])
                score += changepoint_log_prior(V[n], lam=kwargs['lambda_cp'])
            return score

        current_score = total_score(G, V)
        best_V_local = copy.deepcopy(V)
        best_score = current_score

        for step in range(kwargs['num_iters']):
            print(f"  Step {step}...", end="")

            if random.random() < kwargs['p_structure']:
                neighbors = get_graph_neighbors(G, local_nodes, max_parents=kwargs['max_parents'])
                proposal_G = random.choice(neighbors)

                # Detect what changed
                changed_edge = None
                for child in local_nodes:
                    old_parents = set(G[child])
                    new_parents = set(proposal_G[child])
                    if old_parents != new_parents:
                        added = new_parents - old_parents
                        removed = old_parents - new_parents
                        if added:
                            changed_edge = f"Add edge: {list(added)[0]} → {child}"
                        elif removed:
                            changed_edge = f"Remove edge: {list(removed)[0]} → {child}"
                        break

                proposal_score = sum(
                    score_segments_bde(local_df, V[n], n, proposal_G[n], kwargs['alpha']) +
                    changepoint_log_prior(V[n], lam=kwargs['lambda_cp'])
                    for n in local_nodes
                )
                accept_prob = min(1.0, np.exp(proposal_score - current_score))

                print(f" Structure move | {changed_edge or 'Unknown change'} | Score Δ = {proposal_score - current_score:.2f} | Accept Prob = {accept_prob:.3f}")

                if random.random() < accept_prob:
                    G = proposal_G
                    current_score = proposal_score
                    if proposal_score > best_score:
                        print("New BEST graph structure found!")
                        best_score = proposal_score
                        best_V_local = copy.deepcopy(V)

            else:
                node = random.choice(local_nodes)
                move = random.choice(['birth', 'death', 'realloc'])

                if move == 'birth':
                    V_new, ok = birth_move(V[node])
                elif move == 'death':
                    V_new, ok = death_move(V[node])
                else:
                    V_new, ok = reallocation_move(V[node])

                if ok:
                    seg_score_new = score_segments_bde(local_df, V_new, node, G[node], kwargs['alpha'])
                    prior_new = changepoint_log_prior(V_new, lam=kwargs['lambda_cp'])
                    seg_score_old = score_segments_bde(local_df, V[node], node, G[node], kwargs['alpha'])
                    prior_old = changepoint_log_prior(V[node], lam=kwargs['lambda_cp'])

                    proposal_score = seg_score_new + prior_new
                    current_local_score = seg_score_old + prior_old
                    accept_prob = min(1.0, np.exp(proposal_score - current_local_score))

                    print(f"Changepoint move ({move}) on {node} | Δ = {proposal_score - current_local_score:.2f} | Accept Prob = {accept_prob:.3f}")

                    if random.random() < accept_prob:
                        V[node] = V_new
                        current_score = total_score(G, V)
                        if current_score > best_score:
                            print("New BEST changepoint config found!")
                            best_score = current_score
                            best_V_local = copy.deepcopy(V)

            if step % 50 == 0:
                for child, parents in G.items():
                    for parent in parents:
                        edge_counts[(child, parent)] += 1
                total_graph_samples += 1

        for node in local_nodes:
            full_best_V[node][group.index] = best_V_local[node]

        print(f" Done with subject {sid} | Best score: {best_score:.2f}")

    final_graph = defaultdict(list)
    for (child, parent), count in edge_counts.items():
        if count / total_graph_samples >= 0.3:
            final_graph[child].append(parent)

    return final_graph, full_best_V


#Example
columns = [col for col in df_encoded.columns if col not in ['subject_id', 'timestamp']]
best_graph, best_V = run_rjmcmc_per_subject(
    df_encoded,
    nodes=columns,
    num_iters=2000,
    alpha=1.0,
    lambda_cp=5.0,
    max_parents=3,
    p_structure=0.5
)

print("True connections: blood_pressure_t-1 → pain_level, pain_level_t-1 → mobility")
print("Best Graph Structure:")
for child, parents in best_graph.items():
    if parents:
        print(f"{child} <- {parents}")


Running RJMCMC for subject 1...
  Step 0... Structure move | Add edge: pain_level_t-1 → heart_rate_t-1 | Score Δ = 5.21 | Accept Prob = 1.000
New BEST graph structure found!
  Step 1... Structure move | Add edge: mobility_t-1 → mobility | Score Δ = -0.85 | Accept Prob = 0.427
  Step 2... Structure move | Add edge: mobility_t-1 → pain_level | Score Δ = -1.71 | Accept Prob = 0.182
  Step 3... Structure move | Add edge: blood_pressure_t-1 → oxygen_level | Score Δ = -1.31 | Accept Prob = 0.270
  Step 4...Changepoint move (birth) on blood_pressure_t-1 | Δ = -8.23 | Accept Prob = 0.000
  Step 5...  Step 6...  Step 7... Structure move | Add edge: mobility_t-1 → heart_rate | Score Δ = 0.78 | Accept Prob = 1.000
  Step 8...  Step 9...Changepoint move (birth) on mobility_t-1 | Δ = -3.93 | Accept Prob = 0.020
  Step 10...Changepoint move (birth) on mobility | Δ = -4.96 | Accept Prob = 0.007
  Step 11...  Step 12...  Step 13...  Step 14...Changepoint move (birth) on mobility | Δ = -4.69 | Accept P

In [ ]:
def summarize_changepoints(V, df):
    print("Changepoint Summary:")
    for var, Vn in V.items():
        change_locs = [i for i in range(1, len(Vn)) if Vn[i] != Vn[i - 1]]
        timestamps = df.iloc[change_locs]['timestamp'].values if 'timestamp' in df.columns else change_locs
        print(f"- {var}: {len(set(Vn))} segments, {len(change_locs)} changepoints")
        if change_locs:
            print(f"  → Change indices: {change_locs}")
            print(f"  → Change times: {list(timestamps)}")

summarize_changepoints(best_V, df_encoded)



🔍 Changepoint Summary:
- blood_pressure: 2 segments, 2 changepoints
  → Change indices: [5, 19]
  → Change times: [datetime.date(2023, 10, 15), datetime.date(2023, 12, 9)]
- pain_level: 2 segments, 4 changepoints
  → Change indices: [24, 32, 125, 133]
  → Change times: [datetime.date(2023, 12, 19), datetime.date(2023, 1, 10), datetime.date(2023, 7, 29), datetime.date(2023, 11, 19)]
- mobility: 1 segments, 0 changepoints
- heart_rate: 3 segments, 11 changepoints
  → Change indices: [14, 19, 39, 50, 59, 61, 101, 104, 110, 115, 120]
  → Change times: [datetime.date(2023, 11, 2), datetime.date(2023, 12, 9), datetime.date(2023, 1, 24), datetime.date(2023, 12, 14), datetime.date(2024, 1, 1), datetime.date(2023, 9, 28), datetime.date(2023, 3, 7), datetime.date(2023, 3, 13), datetime.date(2023, 4, 10), datetime.date(2023, 4, 20), datetime.date(2023, 7, 19)]
- oxygen_level: 1 segments, 0 changepoints
- blood_pressure_t-1: 2 segments, 2 changepoints
  → Change indices: [5, 19]
  → Change times:

In [ ]:
def truth_changepoints(df, ground_truth_cp):
    print("Ground Truth Changepoints (timestamps per subject):")
    for sid, cp in ground_truth_cp.items():
        ts = df[df['subject_id'] == sid]['timestamp'].reset_index(drop=True)
        if cp < len(ts):
            print(f"Subject {sid}: changepoint at index {cp}, timestamp = {ts[cp]}")

truth_changepoints(df_health, ground_truth_cp)


Ground Truth Changepoints (timestamps per subject):
Subject 1: changepoint at index 10, timestamp = 2023-10-23
Subject 2: changepoint at index 7, timestamp = 2023-12-21
Subject 3: changepoint at index 9, timestamp = 2023-01-26
Subject 4: changepoint at index 6, timestamp = 2023-12-24
Subject 5: changepoint at index 8, timestamp = 2023-10-12
Subject 6: changepoint at index 7, timestamp = 2023-09-11
Subject 7: changepoint at index 10, timestamp = 2023-03-05
Subject 8: changepoint at index 5, timestamp = 2023-04-18
Subject 9: changepoint at index 7, timestamp = 2023-07-31
Subject 10: changepoint at index 6, timestamp = 2023-11-29
